## 移除最后conv最后一层网络

把 resnet最后一层再拼接回去，重新开始训练 ，finetune


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import *
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [14]:
batch_size = 256
num_epochs = 50
extract_size = 2048
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
!mkdir ./bottle_neck/resnet152_train_clear_remove_last_conv

mkdir: cannot create directory ‘./bottle_neck/resnet152_train_clear_remove_last_conv’: File exists


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderClearCut()
encoder = encoder.to(device)

fileFold = './bottle_neck/resnet152_train_clear_remove_last_conv'
data_loader = get_encoder_loader_fold(transform_vaild, encoder, device, fileFold, load=True, mode='train', batch_size=batch_size)

encoding 32738/32739->100.00%, spent_time:52:54.24

### init model

拼接resnet 最后一个 conv layer，用来训练

In [15]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = ConvClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
classify_model.load_state_dict(torch.load('./models/class_train_last_conv_layer_last.pkl'))

### time to train model

In [9]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [ ]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(1, num_epochs+1):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%2 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%10 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_train_last_conv_layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_train_last_conv_layer_last.pkl'))

Epoch [1/50], Step [126/127], Loss: 0.4349, Accuracy: 83.20%, Best_acc: 85.16%            
 Epoch 1, spent time:368.27s
Epoch [2/50], Step [126/127], Loss: 0.2305, Accuracy: 92.58%, Best_acc: 93.75%            
 Epoch 2, spent time:324.95s
Epoch [3/50], Step [126/127], Loss: 0.1583, Accuracy: 94.53%, Best_acc: 96.88%            
 Epoch 3, spent time:261.60s
Epoch [4/50], Step [126/127], Loss: 0.0720, Accuracy: 97.27%, Best_acc: 99.22%            
 Epoch 4, spent time:230.23s
Epoch [5/50], Step [126/127], Loss: 0.0742, Accuracy: 96.48%, Best_acc: 99.61%            
 Epoch 5, spent time:188.45s
Epoch [6/50], Step [126/127], Loss: 0.0447, Accuracy: 98.83%, Best_acc: 99.61%            
 Epoch 6, spent time:152.69s
Epoch [7/50], Step [126/127], Loss: 0.0430, Accuracy: 98.44%, Best_acc: 100.00%             
 Epoch 7, spent time:139.02s
Epoch [8/50], Step [126/127], Loss: 0.0323, Accuracy: 98.83%, Best_acc: 100.00%             
 Epoch 8, spent time:135.37s
Epoch [9/50], Step [126/127], Loss: 

### How good is the model?

In [12]:
!mkdir ./bottle_neck/resnet152_valid_clear_remove_last_conv

In [8]:
fileFold = './bottle_neck/resnet152_valid_clear_remove_last_conv'
valid_data_loader = get_encoder_loader_fold(transform_vaild, encoder, device, fileFold, load=True, mode='valid', batch_size=batch_size)

In [41]:
classify_model.load_state_dict(torch.load('./models/class_train_last_conv_layer_last.pkl'))

In [10]:
classify_model = classify_model.eval()
predict = []
total = len(valid_data_loader.dataset)
for idx in range(total):
    embed, _ = valid_data_loader.dataset[idx]
    p = classify_model(embed).argmax().item()
    predict.append(p)
    print('\r %d / %d' % (idx, total), end='')
    sys.stdout.flush()
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

KeyError: 4982

## With 100 epoch the result is the best,yet.

In [23]:
!mv ./models/class_multify_rm1layer_100.pkl ./models/class_multify_rm1layer_good.pkl